In [1]:
import sys
import os

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../src")))

In [2]:
import pandas as pd

In [3]:
from courses_scheduler.planning import AcademicPlan
from courses_scheduler.combinations import OptionsSet
from courses_scheduler.objects import (
    AcademicDiscipline,
    Classroom,
    Students,
    Teacher,
    TimeSlot,
)

# Подготовка

In [4]:
def make_df(options: OptionsSet, choosen_idx: list[int]) -> pd.DataFrame:
    columns = ["time_slot", "students", "teacher", "classroom", "discipline"]
    res = []
    for i, (ts, s, t, c, d) in enumerate(options):
        if i in choosen_idx:
            res.append((
                ts.__repr__(),
                s.__repr__(),
                t.__repr__(),
                c.__repr__(),
                d.__repr__(),
            ))
    return pd.DataFrame(res, columns=columns)

In [5]:
s = {
    k: Students(group_id=k)
    for k in "ABCDEFGHIJKLMN"
}

In [6]:
t = {
    k: Teacher(name=k)
    for k in "ABCDEFGHIJKLMN"
}

In [7]:
c = {
    k: Classroom(room_number=str(k))
    for k in range(20)
}

In [8]:
d = {
    k: AcademicDiscipline(title=k)
    for k in "ABCDEFGHIJKLMN"
}

In [9]:
ts = {
    k: TimeSlot(date_from=k)
    for k in range(20)
}

# Простейший сценарий

In [29]:
ap = AcademicPlan(
    students_workload={s["A"]: {d["A"]: 1}},
    teachers_workload={t["A"]: {d["A"]: 1}},
    available_classrooms={c[1], c[2]},
    available_time_slots=ts,
)

ap.optimizer.model.solve(disp=False)

In [30]:
df = make_df(ap.optimizer.options, ap.optimizer.choosen_options_idx)
df.head()

,time_slot,students,teacher,classroom,discipline
0,1,A,A,1,A


# Несколько учебных групп

In [27]:
ap = AcademicPlan(
    students_workload={s["A"]: {d["A"]: 1}, s["B"]: {d["A"]: 1}},
    teachers_workload={t["A"]: {d["A"]: 2}},
    available_classrooms={c[1], c[2]},
    available_time_slots=ts,
)

ap.optimizer.model.solve(disp=False)

In [28]:
df = make_df(ap.optimizer.options, ap.optimizer.choosen_options_idx)
df.head()

,time_slot,students,teacher,classroom,discipline
0,1,B,A,2,A
1,2,A,A,1,A


# Несколько преподавателей

In [25]:
ap = AcademicPlan(
    students_workload={s["A"]: {d["A"]: 1}},
    teachers_workload={t["A"]: {d["A"]: 1}, t["B"]: {d["A"]: 1}},
    available_classrooms={c[1], c[2]},
    available_time_slots=ts,
)

ap.optimizer.model.solve(disp=False)

In [26]:
df = make_df(ap.optimizer.options, ap.optimizer.choosen_options_idx)
df.head()

,time_slot,students,teacher,classroom,discipline
0,19,A,B,1,A


# Несколько дисциплин

In [23]:
ap = AcademicPlan(
    students_workload={s["A"]: {d["A"]: 1, d["B"]: 1}},
    teachers_workload={t["A"]: {d["A"]: 1, d["B"]: 1}},
    available_classrooms={c[1], c[2]},
    available_time_slots=ts,
)

ap.optimizer.model.solve(disp=False)

In [24]:
df = make_df(ap.optimizer.options, ap.optimizer.choosen_options_idx)
df.head()

,time_slot,students,teacher,classroom,discipline
0,4,A,A,2,B
1,19,A,A,1,A


# Несколько предметов у разных групп

In [31]:
ap = AcademicPlan(
    students_workload={s["A"]: {d["A"]: 1}, s["B"]: {d["B"]: 1}},
    teachers_workload={t["A"]: {d["A"]: 1, d["B"]: 1}},
    available_classrooms={c[1], c[2]},
    available_time_slots=ts,
)

ap.optimizer.model.solve(disp=False)

In [32]:
df = make_df(ap.optimizer.options, ap.optimizer.choosen_options_idx)
df.head()

,time_slot,students,teacher,classroom,discipline
0,4,B,A,2,B
1,19,A,A,1,A


# Нехватка ресурсов

In [33]:
ap = AcademicPlan(
    students_workload={s["A"]: {d["A"]: 2}},
    teachers_workload={t["A"]: {d["A"]: 1}},
    available_classrooms={c[1], c[2]},
    available_time_slots=ts,
)

ap.optimizer.model.solve(disp=False)

Exception: @error: Solution Not Found


# Большие объемы

In [10]:
ts_large = {
    k: TimeSlot(date_from=k)
    for k in range(500)
}

In [11]:
%%time
ap = AcademicPlan(
    students_workload={s_i: {d_i: 10 for d_i in d} for s_i in s},
    teachers_workload={t_i: {d_i: 2 for d_i in d} for t_i in t},
    available_classrooms=c.values(),
    available_time_slots=ts_large,
)

2024-09-21 19:31:18.649 | INFO     | courses_scheduler.planning:build_optimizer:71 - Start building optimizer
2024-09-21 19:31:18.650 | INFO     | courses_scheduler.planning:build_options:51 - Start building options
100%|██████████| 14/14 [00:11<00:00,  1.26it/s]
2024-09-21 19:31:37.596 | INFO     | courses_scheduler.optimization:__init__:24 - Start declaring optimized vars
  9%|▉         | 2486909/27440000 [00:16<02:03, 202627.28it/s]

KeyboardInterrupt: 

  9%|▉         | 2486909/27440000 [00:31<02:03, 202627.28it/s]

In [ ]:
%%time
ap.optimizer.model.solve(disp=False)